This code is used to classify the M5S users, training the algorithm with label left and right users. 

In [0]:
# Install the PyDrive wrapper & import libraries.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
listed = drive.ListFile({'q': "title contains '.pkl' and 'root' in parents"}).GetList()
for file in listed:
    print('title {}, id {}'.format(file['title'], file['id']))

title data_m5s_cleaned.pkl, id 1ibuI1n_LfAkZSzzXHIA5GKXmARyTO0de
title data_75_fasttext.pkl, id 1i4vviPsX50thW6KTg_x8p41BlmvAkng5
title refined_giacomo_100.pkl, id 107988YA16LXnv_b1cTkyf8-wqFlVnMsB
title refined_giacomo_75.pkl, id 1Uxbiwkwt3Wt1vH-vZfD6k36J8SMBJHrz
title refined1.pkl, id 1_-xtFh-iPf2-prHJeNZnOuR_y0WZlPLZ
title base_df.pkl, id 1Q1HYn8Xgp0azDuzFoSmRggg3GcWuizF3


In [0]:
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

import io
import pickle
from googleapiclient.http import MediaIoBaseDownload

file_id = '107988YA16LXnv_b1cTkyf8-wqFlVnMsB'

request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    _, done = downloader.next_chunk()

downloaded.seek(0)
f = pickle.load(downloaded)
df = f

In [0]:
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

import io
import pickle
from googleapiclient.http import MediaIoBaseDownload

file_id = '1ibuI1n_LfAkZSzzXHIA5GKXmARyTO0de'

request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    _, done = downloader.next_chunk()

downloaded.seek(0)
h = pickle.load(downloaded)
df_5s = h

In [0]:
import pandas as pd
import re
df["tweets"]=df["tweets"].apply(lambda s: ' '.join(re.sub("rt", "", s).split()))
df_5s["tweets"]=df_5s["tweets"].apply(lambda s: ' '.join(re.sub("rt", "", s).split()))
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [0]:
#  Create a list of of lists containing the words for each tweet for the two groups
words_in_tweet_rl = [tweet.split() for tweet in df['tweets']]
words_in_tweet_5s = [tweet.split() for tweet in df_5s['tweets']]

In [0]:
# libraries to go on..
import itertools
import collections
# List of all words across tweets
all_words_rl = list(itertools.chain(*words_in_tweet_rl))
all_words_5s = list(itertools.chain(*words_in_tweet_5s))

In [0]:
tot_voc=list(set(all_words_rl) & set(all_words_5s))

In [0]:
def keep_rl(input_text):
    words = input_text.split()
    noise_free_words = [word for word in words if word in tot_voc] 
    noise_free_text = " ".join(noise_free_words) 
    return noise_free_text

In [0]:
df["tweets"] = df["tweets"].apply(lambda s: keep_rl(s))
df_5s["tweets"] = df_5s["tweets"].apply(lambda s: keep_rl(s))

In [0]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(df['tweets'],df['__label__'],test_size=0.2)

In [0]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [0]:
df["tweets"] = df["tweets"].apply(lambda s: str(s))

Tfidf_vect = TfidfVectorizer(ngram_range=(1,1))
Tfidf_vect.fit(df['tweets'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [0]:
# Classifier - Algorithm - SVM
# Fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# Predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

In [0]:
test_5s = df_5s.tweets

Tfidf_vect1 = TfidfVectorizer(ngram_range=(1,1))
Tfidf_vect1.fit(df_5s['tweets'])
Test_5s_Tfidf = Tfidf_vect1.transform(test_5s)

In [0]:
predictions_5s = SVM.predict(Test_5s_Tfidf)

In [0]:
prediction=list(predictions_5s)
right=prediction.count(0)
left=prediction.count(1)
right_prop=right/len(prediction)
left_prop=left/len(prediction)